# Shopify Backend Intern Challenge

In [1]:
import requests
import json
from pprint import pprint

In [2]:
API_URL = "https://backend-challenge-summer-2018.herokuapp.com/challenges.json?id=1&page=1" 
split_URL = API_URL.rpartition("=")

In [3]:
def fetch(url): 
    r = requests.get(url)

    # will throw an error for status > 400
    r.raise_for_status()

    return r.json()

In [4]:
response = fetch(API_URL)
total = response['pagination']['total']
max_pages =  total / response['pagination']['per_page']

# account for non-full pages
if max_pages % 1 != 0:
    max_pages += 1

In [5]:
unflattened_nodes = []
page = 1

while (page <= max_pages):
    unflattened_nodes.append(response['menus'])
    page += 1
    
#   make another call
    response = fetch(split_URL[0] + split_URL[1] + str(page))
    
nodes = [item for sublist in unflattened_nodes for item in sublist]

In [6]:
# recurses up to find root node
# because max depth = 4, O(n) ~= Const
def find_root(node_id, nodes):

    if ('parent_id' in nodes[node_id -1]):
        # assumes each node has at most one parent
        
        return find_root(nodes[node_id - 1]['parent_id'], nodes)
    
    else:
        return node_id
         

In [7]:
# make set of all root nodes 
roots = set()
for node in nodes:
    root = find_root(node['id'], nodes)
    roots.add(root)

In [8]:
# recursively search for all children nodes
def map_children(node_id, nodes, all_children= []): 
    node_children = nodes[node_id - 1]['child_ids']
    
    # If empty child array 
    if (not node_children):
        return 
    
    else:
        for child in node_children:
            if child not in all_children:
                all_children.append(child)
                map_children(child, nodes, all_children)

    return all_children   
              

In [9]:
checked = []
output = {
    
    "valid_menus": [],

    "invalid_menus": [],
}

for root in roots:
    
    # find all children of root
    children = map_children(root, nodes, all_children=[])

    # validate
    valid = (root not in children)

    # add to output schema
    if (valid):
        output["valid_menus"].append( {"root_id": root, "children": children})
    else:
        output["invalid_menus"].append( {"root_id": root, "children": children})



In [10]:
pprint(output)

{'invalid_menus': [{'children': [3, 7, 15, 1], 'root_id': 1}],
 'valid_menus': [{'children': [4, 5, 6, 8], 'root_id': 2},
                 {'children': [10, 11, 12, 13, 14], 'root_id': 9}]}


# Extra challenge

For the extra challenge simply change `API_URL` to: 

`"https://backend-challenge-summer-2018.herokuapp.com/challenges.json?id=2&page=1"` 

and re-run. This will output the following:


```
{'invalid_menus': [{'children': [3, 5, 18, 4, 6, 7, 20, 1, 19], 'root_id': 1}],
 'valid_menus': [{'children': [8, 9, 10, 11], 'root_id': 2},
                 {'children': [13, 14, 15, 16, 17, 21], 'root_id': 12}]}
```